In [2]:
import os
import pandas as pd

GENERAL_DATA_PATH = os.path.join("Datasets", "general_data", "general_data.csv")
MANAGER_SURVEY_DATA_PATH = os.path.join("Datasets", "manager_survey_data")
EMPLOYEE_SURVEY_DATA_PATH = os.path.join("Datasets", "employee_survey_data",)

def load_general_data(general_path=GENERAL_DATA_PATH):
    csv_path = os.path.join(general_path, "general_data.csv")
    return pd.read_csv(csv_path)

def load_manager_survey_data(manager_survey_data=GENERAL_DATA_PATH):
    csv_path = os.path.join(manager_survey_data, "manager_survey_data.csv")
    return pd.read_csv(csv_path)

def load_employee_survey_data(employee_survey_data=GENERAL_DATA_PATH):
    csv_path = os.path.join(employee_survey_data, "employee_survey_data.csv")
    return pd.read_csv(csv_path)

general = load_general_data()
manager_survey = load_manager_survey_data()
employee_survey = load_employee_survey_data()